In [2]:
import numpy as np
import re

In [ ]:
re.compile("^([A-Z][0-9]+)+$")

In [7]:
teste_int = '12354'

In [43]:
PADRAO_INICIO = re.compile('[0-9/+/-]')

In [44]:
PADRAO_INICIO.search('a')

In [14]:
PADRAO_INICIO.(teste_int)[0]

'12354'

In [24]:
teste = re.compile('')

In [29]:
teste.search('1')

<re.Match object; span=(0, 0), match=''>

In [75]:
DEFINICAO_AUTOMATO = {
    'completo' :{

        'estado_inicial':'Q0',

        'estados_finais' : {
            'QD1':'Variável', 
            'QB4': 'Comentário', 
            'QA2': 'Comentário', 
            'QC2': 'Comentário', 
            'QE1': 'Numérico Inteiro', 
            'QE3': 'Numérico Float'
            },
            
        'Q0' : {
            'mudanca_estado':['[A-Za-z]', '[(]', '[/]', '[{]', '[0-9/+/-]'],
            'prox_estado':['QD1', 'QB1', 'QA1', 'QC1', 'QE1']
        },
        'QA1' : {
            'mudanca_estado':['[/]'],
            'prox_estado':['QA2']
        },
        'QA2' : {
            'mudanca_estado':['[^\n]'],
            'prox_estado':['QA2']
        },
        'QB1': {
            'mudanca_estado':['[*]'],
            'prox_estado':['QB2']
        },
        'QB2': {
            'mudanca_estado':['[^\n\*]', '[*]'],
            'prox_estado':['QB2', 'QB3']
        },
        'QB3': {
            'mudanca_estado':['[^\n\)]', '[)]'],
            'prox_estado':['QB2', 'QB4']
        }        ,
        'QC1': {
            'mudanca_estado':['[^\n}]', '[}]'],
            'prox_estado':['QC1', 'QC2']
        },
        'QD1': {
            'mudanca_estado':['[A-Za-z0-9\_]'],
            'prox_estado':['QD1']
        },
        'QE1': {
            'mudanca_estado':['[0-9]', '[.]'],
            'prox_estado':['QE1', 'QE2']
        },
        'QE2': {
            'mudanca_estado':['[0-9]'],
            'prox_estado':['QE3']
        },
        'QE3': {
            'mudanca_estado':['[0-9]'],
            'prox_estado':['QE3']
        }

    },
    'automato_intierios' :{

        'estado_inicial':'Q0',
        'estados_finais' : ['QA1'],
        'classe': 'Numérico Inteiro',
        'Q0' : {
            'mudanca_estado':['[0-9/+/-]'],
            'prox_estado':['QA1']
        },
        'QA1' : {
            'mudanca_estado':['[0-9]'],
            'prox_estado':['QA1']
        }
    },
    'automato_float' :{

        'estado_inicial':'Q0',
        'estados_finais' : ['QA3'],
        'classe': 'Numérico Float',
        'Q0' : {
            'mudanca_estado':['[0-9/+/-]'],
            'prox_estado':['QA1']
        },
        'QA1' : {
            'mudanca_estado':['[0-9]', '[.]'],
            'prox_estado':['QA1', 'QA2']
        },
        'QA2' : {
            'mudanca_estado':['[0-9]'],
            'prox_estado':['QA3']
        },
        'QA3': {
            'mudanca_estado':['[0-9]'],
            'prox_estado':['QA3']
        }
    },

    'variavel' :{

        'estado_inicial':'Q0',
        'estados_finais' : ['Q1'],
        'classe': 'Variável',

        'Q0' : {
            'mudanca_estado':['[A-Za-z]'],
            'prox_estado':['Q1']
        },
        'Q1' : {
            'mudanca_estado':['[A-Za-z0-9\_]'],
            'prox_estado':['Q1']
        }
    },
    'comentario' :{

        'estado_inicial':'Q0',
        'estados_finais' : ['QA2', 'QB4', 'QC2'],
        'classe': 'Comentário',

        'Q0' : {
            'mudanca_estado':['[(]', '[/]', '[{]'],
            'prox_estado':['QB1', 'QA1', 'QC1']
        },
        'QA1' : {
            'mudanca_estado':['[/]'],
            'prox_estado':['QA2']
        },
        'QA2' : {
            'mudanca_estado':['[^\n]'],
            'prox_estado':['QA2']
        },
        'QB1': {
            'mudanca_estado':['[*]'],
            'prox_estado':['QB2']
        },
        'QB2': {
            'mudanca_estado':['[^\n\*]', '[*]'],
            'prox_estado':['QB2', 'QB3']
        },
        'QB3': {
            'mudanca_estado':['[^\n\)]', '[)]'],
            'prox_estado':['QB2', 'QB4']
        }        ,
        'QC1': {
            'mudanca_estado':['[^\n}]', '[}]'],
            'prox_estado':['QC1', 'QC2']
        }
    }


}

In [81]:
def imprime_mudaca_estado(estado_atual, entrada, proximo_estado):

    saida = 'Estado Atual: {}; Entrada: {}, Próximo Estado: {}'.format(estado_atual, entrada, proximo_estado)
    print(saida)

def processa_entrada(valor_entrada, automato = DEFINICAO_AUTOMATO['completo']):

    valor_entrada = str(valor_entrada)
    estado_atual = automato['estado_inicial']

    try:
        for c in valor_entrada:
            mudancas_estado = automato[estado_atual].get('mudanca_estado')
            idx_mudanca_estado = 0
            for mudanca_estado in mudancas_estado:

                padrao = re.compile(mudanca_estado)

                if padrao.search(c):
                    proximos_estados = automato[estado_atual].get('prox_estado')
                    proximo_estado = proximos_estados[idx_mudanca_estado]

                    imprime_mudaca_estado(estado_atual, c,  proximo_estado)
                    estado_atual = proximo_estado
                    break

                idx_mudanca_estado += 1

                if idx_mudanca_estado >= len(mudancas_estado):
                    raise Exception("Entrada Inválida --> {}".format(c))
        
        classe = automato['estados_finais'].get(estado_atual)
        if classe:
            print('Entrada Aceita com Sucesso para a Classe {}'.format(classe))
        else:
            raise Exception("Entrada Inválida".format(c))
    except Exception as e:
        print(e)

In [90]:
processa_entrada(valor_entrada='454564564')

Estado Atual: Q0; Entrada: 4, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 5, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 4, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 5, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 6, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 4, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 5, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 6, Próximo Estado: QE1
Estado Atual: QE1; Entrada: 4, Próximo Estado: QE1
Entrada Aceita com Sucesso para a Classe Numérico Inteiro
